# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Lab 13** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

**Integrantes**:
- Lorena Ruelas Gaytán
- Yael Alejandro Rodríguez Barreto
- Ximena Isaac Horta
- Alberto Renteria Camacho

In [8]:
import findspark
findspark.init()

#### Spark Conexion

In [9]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MLSpark-K-means") \
    .master("spark://3cf8f1f4c83b:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext
spark.conf.set("spark.sql.shuffle.partitions", "5")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/10 17:07:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### Data preparation

In [10]:
from team_name.spark_utils import SparkUtils

csv_path = "/home/jovyan/notebooks/data/wines_data/wine-clustering.csv"

headers = [ "Alcohol", "Malic_Acid", "Ash", "Ash_Alcanity",
        "Magnesium", "Total_Phenols", "Flavanoids", "Nonflavanoid_Phenols",
        "Proanthocyanins", "Color_Intensity", "Hue", "OD280", "Proline"
]

schema = SparkUtils.generate_schema([(head, "float") for head in headers])

# Convert list to a DataFrame
wines_df = spark.read \
        .schema(schema) \
        .option("header", "true") \
        .csv(csv_path)

wines_df = wines_df.dropna(how="any")
wines_df.show(10)

+-------+----------+----+------------+---------+-------------+----------+--------------------+---------------+---------------+----+-----+-------+
|Alcohol|Malic_Acid| Ash|Ash_Alcanity|Magnesium|Total_Phenols|Flavanoids|Nonflavanoid_Phenols|Proanthocyanins|Color_Intensity| Hue|OD280|Proline|
+-------+----------+----+------------+---------+-------------+----------+--------------------+---------------+---------------+----+-----+-------+
|  14.23|      1.71|2.43|        15.6|    127.0|          2.8|      3.06|                0.28|           2.29|           5.64|1.04| 3.92| 1065.0|
|   13.2|      1.78|2.14|        11.2|    100.0|         2.65|      2.76|                0.26|           1.28|           4.38|1.05|  3.4| 1050.0|
|  13.16|      2.36|2.67|        18.6|    101.0|          2.8|      3.24|                 0.3|           2.81|           5.68|1.03| 3.17| 1185.0|
|  14.37|      1.95| 2.5|        16.8|    113.0|         3.85|      3.49|                0.24|           2.18|            7.

### Assemble the features into a single vector column

In [11]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=headers, outputCol="features")
assembled_df = assembler.transform(wines_df)

---
# KMeans at the same time

#### Initialize KMeans

In [12]:
from pyspark.ml.clustering import KMeans

k_values = [2, 5, 10, 15, 20]
kmeans = [KMeans().setK(k).setSeed(19) for k in k_values]

#### Training

In [13]:
models = [k.fit(assembled_df) for k in kmeans]

25/05/10 17:08:53 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/05/10 17:09:07 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/05/10 17:09:07 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


#### Predictions

In [14]:
predictions = [model.transform(assembled_df) for model in models]

#### Evaluate model

In [15]:
from pyspark.ml.evaluation import ClusteringEvaluator

for i in range(len(k_values)):
    print(f"\n K values = {k_values[i]}")
    evaluator = ClusteringEvaluator()
    silhouette = evaluator.evaluate(predictions[i])
    print(f"Silhouette score: {silhouette}")

    


 K values = 2


Silhouette score: 0.821360351333219

 K values = 5
Silhouette score: 0.730925679074768

 K values = 10
Silhouette score: 0.6099911274077665

 K values = 15
Silhouette score: 0.6663309347635443

 K values = 20
Silhouette score: 0.6341822740902026


In [16]:
for i in range(len(k_values)):
    print(f"\n K values = {k_values[i]}")
    print("Cluster Centers: ")
    for center in models[i].clusterCenters():
        print(center)


 K values = 2
Cluster Centers: 
[1.27028455e+01 2.54455285e+00 2.33910569e+00 2.04081301e+01
 9.68130081e+01 2.06211382e+00 1.64146342e+00 3.92682924e-01
 1.45406503e+00 4.85138211e+00 9.08617886e-01 2.40821138e+00
 5.65869919e+02]
[1.36665455e+01 1.87072727e+00 2.42781818e+00 1.74527272e+01
 1.06290909e+02 2.81618182e+00 2.89654548e+00 2.92909090e-01
 1.89690911e+00 5.52036361e+00 1.06665455e+00 3.06672727e+00
 1.15172727e+03]

 K values = 5
Cluster Centers: 
[1.25042424e+01 2.44318182e+00 2.28378789e+00 2.07772727e+01
 9.24696970e+01 2.07333332e+00 1.79545455e+00 3.84696969e-01
 1.47181817e+00 4.07242422e+00 9.46212124e-01 2.50484848e+00
 4.52545455e+02]
[1.29450000e+01 2.64017857e+00 2.40678570e+00 2.00071429e+01
 1.01660714e+02 2.06107143e+00 1.47535715e+00 4.03928567e-01
 1.43874999e+00 5.70946429e+00 8.68928568e-01 2.30160714e+00
 6.94267857e+02]
[1.41366666e+01 1.83166669e+00 2.41166667e+00 1.62666666e+01
 1.07666667e+02 3.25499996e+00 3.49333338e+00 2.71666663e-01
 2.21666672e

In [17]:
sc.stop()